In [ ]:
from google.colab import drive

drive.mount ('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import requests

# Replace 'your_api_key' with your actual Google Geocoding API key
API_KEY = '<REPLACE WITH YOUR API_KEY>'
#Main Parking Signage Csv File URL
CSV_FILE = 'https://raw.githubusercontent.com/quanghuynguyenhua/Montreal-Parking/main/assets/signalisation_stationnement.csv'

def reverse_geocode(lat, lng):
    """
    Perform reverse geocoding to get the JSON response and extract the street number and street name
    from latitude and longitude values.
    """
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'latlng': f"{lat},{lng}",
        'key': API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        json_response = response.json()
        if json_response['results']:
            address_components = json_response['results'][0]['address_components']
            # Initialize variables for street number and street name
            street_number = None
            street_name = None
            # Iterate through address components to find street number and street name
            for component in address_components:
                if 'street_number' in component['types']:
                    street_number = component['long_name']
                elif 'route' in component['types']:
                    street_name = component['long_name']
            return street_number, street_name
        else:
            return None, None
    else:
        return None, None

In [ ]:
#Test function:
reverse_geocode(45.547944, -73.571238)

('5101', '7e Avenue')

In [ ]:
import time

def main():
    df = pd.read_csv(CSV_FILE)
    chunk_size = 3000
    total_rows = len(df)

    for start_row in range(0, total_rows, chunk_size):
        end_row = min(start_row + chunk_size, total_rows)
        df_chunk = df.iloc[start_row:end_row]

        street_numbers = []
        street_names = []

        for index, row in df_chunk.iterrows():
            lat = row['Latitude']
            lng = row['Longitude']
            street_number, street_name = reverse_geocode(lat, lng)
            street_numbers.append(street_number)
            street_names.append(street_name)

        df.loc[start_row:end_row-1, 'street_number'] = street_numbers
        df.loc[start_row:end_row-1, 'street_name'] = street_names

        print(f"Processed rows {start_row} to {end_row-1}.")

        if end_row != total_rows:
            print("Waiting for 60 seconds before processing the next chunk...")
            time.sleep(60)


    # Optionally, save the modified DataFrame to a new Excel file
    df.to_csv('signalisation_stationnement_with_addresses.csv', index=False)


# if __name__ == "__main__":
#     main()

DO REVERSE GEOCOING TO CREATE A CSV FILE HOLDING ADDRESS

In [ ]:
main()

Processed rows 0 to 2999.
Waiting for 60 seconds before processing the next chunk...
Processed rows 3000 to 5999.
Waiting for 60 seconds before processing the next chunk...
Processed rows 6000 to 8999.
Waiting for 60 seconds before processing the next chunk...
Processed rows 9000 to 11999.
Waiting for 60 seconds before processing the next chunk...
Processed rows 12000 to 14999.
Waiting for 60 seconds before processing the next chunk...
Processed rows 15000 to 17999.
Waiting for 60 seconds before processing the next chunk...
Processed rows 18000 to 18761.


In [ ]:
#Copy the new table with address to the drive folder
!scp -r /content/signalisation_stationnement_with_addresses.csv '/content/gdrive/MyDrive/montreal-parking/code'

BELOW IS THE PART OF CREATE ANOTHER TABLE HOLDING CONNECTED PATHS.

In [ ]:
import pandas as pd
import numpy as np
import re

# Load the data
df = pd.read_csv('signalisation_stationnement_with_addresses.csv')

# Define a function to extract the first number from the street number
def extract_first_number(s):
    # Find all sequences of digits
    numbers = re.findall(r'\d+', s)
    # Return the first number as integer, or return np.nan if there are no numbers
    return int(numbers[0]) if numbers else np.nan

# Apply the function to the 'street_number' column to extract the first number
df['first_number'] = df['street_number'].apply(lambda x: extract_first_number(str(x)))

# Calculate parity (even or odd) of the extracted number
df['parity'] = df['first_number'] % 2

df


,POTEAU_ID_POT,POSITION_POP,PANNEAU_ID_PAN,PANNEAU_ID_RPA,DESCRIPTION_RPA,CODE_RPA,FLECHE_PAN,TOPONYME_PAN,DESCRIPTION_CAT,POTEAU_VERSION_POT,...,DESCRIPTION_RTP,X,Y,Longitude,Latitude,NOM_ARROND,street_number,street_name,first_number,parity
0,3413,4,1785363,11,\A EN TOUT TEMPS,AD-TT,0,NaN,STATIONNEMENT,7,...,10- Fût et feux,300714.800,5043747.000,-73.552305,45.533627,Ville-Marie,NaN,NaN,NaN,NaN
1,3416,4,1762438,14077,\P 12h30-13h30 MAR. 1 AVRIL AU 1 DEC.,SU-MK-A,0,NaN,STATIONNEMENT,10,...,4- Poteau en bois,300497.531,5043851.500,-73.555088,45.534566,Ville-Marie,NaN,NaN,NaN,NaN
2,3419,7,1793563,2365,\P 09h-22h30,SD-JP,3,NaN,STATIONNEMENT,5,...,2- Tige et manchon,300534.969,5043848.000,-73.554608,45.534535,Ville-Marie,2148,Rue du Havre,2148.0,0.0
3,3419,6,1793564,14058,\P 12h30-13h30 LUN. 1 AVRIL AU 1 DEC.,SU-LK-A,0,NaN,STATIONNEMENT,5,...,2- Tige et manchon,300534.969,5043848.000,-73.554608,45.534535,Ville-Marie,2148,Rue du Havre,2148.0,0.0
4,3421,7,1770102,2365,\P 09h-22h30,SD-JP,2,NaN,STATIONNEMENT,5,...,1- Tige et base,300577.313,5043824.500,-73.554066,45.534324,Ville-Marie,2567,Rue Larivière,2567.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18757,297746,7,1863351,14076,\P 12h-13h MAR. 1 AVRIL AU 1 DEC.,SU-MJ-A,0,NaN,STATIONNEMENT,2,...,4- Poteau en bois,300345.441,5044079.117,-73.557037,45.536613,Ville-Marie,2354,Rue Bercy,2354.0,0.0
18758,297747,7,1863352,14057,\P 12h-13h LUN. 1 AVRIL AU 1 DEC.,SU-LJ-A,0,NaN,STATIONNEMENT,2,...,4- Poteau en bois,300348.091,5044091.507,-73.557003,45.536725,Ville-Marie,2383,Rue Bercy,2383.0,1.0
18759,297748,7,1863354,14076,\P 12h-13h MAR. 1 AVRIL AU 1 DEC.,SU-MJ-A,0,NaN,STATIONNEMENT,2,...,4- Poteau en bois,300316.192,5044093.942,-73.557412,45.536747,Ville-Marie,2694,Rue Hochelaga,2694.0,0.0
18760,297760,7,1864029,16338,\A 06h-10h ET 14h30-19h30 LUN A VEN,AV-RR,0,NaN,STATIONNEMENT,2,...,3- Fût,298948.599,5039367.058,-73.574866,45.494202,Ville-Marie,1153,Rue Guy,1153.0,1.0


In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Convert 'street_number' to a numeric type, setting errors='coerce' will turn errors into NaN
df['street_number_numeric'] = pd.to_numeric(df['street_number'], errors='coerce')

# Filter for FLECH_PAN values of 2 and 3 only
df_filtered = df[df['FLECHE_PAN'].isin([2, 3])]
df_filtered = df.dropna(subset=['street_number_numeric'])
df_filtered['street_number_numeric'] = df_filtered['street_number_numeric'].astype(int)

# Define a function to calculate distance based on longitude and latitude
def calculate_distance(group):
    # Extract the coordinates
    coords = group[['Latitude', 'Longitude']].values
    # Calculate the distance matrix
    distance_matrix = cdist(coords, coords, metric='euclidean')
    # Initialize columns for DEST, DEST_LAT, and DEST_LON if they don't exist
    if 'DEST' not in group.columns:
        group['DEST'] = np.nan
    if 'DEST_LAT' not in group.columns:
        group['DEST_LAT'] = np.nan
    if 'DEST_LON' not in group.columns:
        group['DEST_LON'] = np.nan
    # For each row in the group, find the index of the minimum distance that is not zero (i.e., not the point itself)
    for i in range(len(group)):
        # Ignore the distance to itself by setting it to np.inf
        distance_matrix[i, i] = np.inf
        # Find the index of the minimum distance
        min_dist_idx = distance_matrix[i].argmin()
        # Find the original index of the nearest point
        original_idx = group.iloc[min_dist_idx].name
        # Assign the POTEAU_ID_POT of the nearest point
        group.iloc[i, group.columns.get_loc('DEST')] = group.loc[original_idx, 'POTEAU_ID_POT']
        group.iloc[i, group.columns.get_loc('DEST_LAT')] = group.loc[original_idx, 'Latitude']
        group.iloc[i, group.columns.get_loc('DEST_LON')] = group.loc[original_idx, 'Longitude']
    return group


# Create a new column to indicate the parity of the 'street_number_numeric'
df_filtered['street_number_parity'] = df_filtered['street_number_numeric'] % 2

# perform the grouping operation with the filtered and processed dataframe
df_grouped = df_filtered.groupby(['CODE_RPA', 'street_name', 'street_number_parity']).apply(calculate_distance)

# set the ORIGIN columns to the current rows' IDs and coordinates
df_grouped['ORIGIN'] = df_grouped['POTEAU_ID_POT']
df_grouped['ORIGIN_LAT'] = df_grouped['Latitude']
df_grouped['ORIGIN_LON'] = df_grouped['Longitude']

# Remove rows where ORIGIN is the same as DEST
df_grouped = df_grouped[df_grouped['ORIGIN'] != df_grouped['DEST']]

# Sort each pair to create a unique identifier for the pair, regardless of direction
df_grouped['PAIR_ID'] = df_grouped.apply(lambda row: tuple(sorted([row['ORIGIN'], row['DEST']])), axis=1)

# Drop duplicates based on the new PAIR_ID
df_grouped = df_grouped.drop_duplicates(subset=['PAIR_ID'])

# Drop the PAIR_ID column if it's no longer needed
df_grouped.drop('PAIR_ID', axis=1, inplace=True)

# Reorder or select the columns as needed
df_final = df_grouped[['ORIGIN', 'DEST', 'ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON']]

# Reset the index with .reset_index() to turn the index back into regular columns
df_final = df_final.reset_index(drop=False)

<ipython-input-7-ae9fcbfcd76d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['street_number_numeric'] = df_filtered['street_number_numeric'].astype(int)
<ipython-input-7-ae9fcbfcd76d>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['street_number_parity'] = df_filtered['street_number_numeric'] % 2
<ipython-input-7-ae9fcbfcd76d>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [ ]:
df_final

,CODE_RPA,street_name,street_number_parity,level_3,ORIGIN,DEST,ORIGIN_LAT,ORIGIN_LON,DEST_LAT,DEST_LON
0,AD-CD,Rue Saint-Paul East,0,10138,119619,119621.0,45.507644,-73.552316,45.507545,-73.552388
1,AD-DC,Chemin Remembrance,0,9191,104294,104295.0,45.500577,-73.598867,45.500784,-73.598606
2,AD-DC,Chemin Remembrance,0,9195,104300,104295.0,45.501014,-73.598302,45.500784,-73.598606
3,AD-DC,Chemin Remembrance,0,9204,104334,104343.0,45.501960,-73.596515,45.502082,-73.596111
4,AD-DC,Chemin Remembrance,0,9209,104343,236862.0,45.502082,-73.596111,45.502195,-73.595805
...,...,...,...,...,...,...,...,...,...,...
7657,SV-VA-A,Rue Ontario East,1,16097,239313,30155.0,45.530278,-73.555361,45.530730,-73.554930
7658,SV-VA-A,Rue Saint-Paul East,0,7621,92566,92569.0,45.509354,-73.551537,45.509516,-73.551459
7659,SX-JF,Rue Dorion,0,10766,130927,130928.0,45.525460,-73.555146,45.525262,-73.554702
7660,SX-JF,Rue Dorion,1,10466,124011,130952.0,45.524404,-73.552877,45.524606,-73.553320


In [ ]:
df_final.to_excel('path_RPA_signalisation.xlsx', index=True)

#Copy the path table to the drive folder
!scp -r /content/path_RPA_signalisation.xlsx '/content/gdrive/MyDrive/montreal-parking/code'